In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R
# import gtsam

In [3]:
###################################################################
#####  PROCESS the drift error from checkerboard              ##### 
###################################################################
import numpy as np
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.spatial.transform import Rotation as R
import cv2


In [4]:
def draw(img, corners, imgpts):
    corners = corners.astype(np.int32)
    imgpts = imgpts.astype(np.int32)
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (0,0,255), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (255,0,0), 5)
    return img

In [5]:
def getPoseFromChessBoard(img_inp, k, dist, squarSize):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    objp = np.zeros((16*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:16,0:9].T.reshape(-1,2)
    objp = objp * squarSize
    axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
    axis = axis * squarSize
    T_ci = np.zeros((4,4), np.float64)
    ## detect chesssboard
    ret, corners = cv2.findChessboardCorners(img_inp, (16,9),None, cv2.CALIB_CB_FAST_CHECK)
   
    if ret == True:
        print("corners found")
        corners2 = cv2.cornerSubPix(img_inp,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        _,rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, k, dist)
        rot_mat, _ = cv2.Rodrigues(rvecs)
        ## get pose WRT chessboard T_ci
        T_ic = np.vstack((np.hstack((rot_mat, tvecs)), np.array([0, 0, 0, 1])))
        T_ci = np.linalg.inv(T_ic)
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, k, dist)
        cv2.drawChessboardCorners(img_inp, (16,9), corners2, ret)
        img = cv2.cvtColor(img_inp,cv2.COLOR_GRAY2RGB)
        img = draw(img,corners2,imgpts)
#         cv2.imshow('img',img)
#         cv2.waitKey(20)
    else:
        print("Corners not found")
    return T_ci

In [14]:
def computeError(df_traj, tStamp, k, dist, squareSize, patternSize, T_c0=np.eye(4)):
    img_gray = cv2.imread(os.path.join(data_path,str(tStamp)+".jpg"), 0)
    #clahe = cv2.createCLAHE(clipLimit = 5)
    #img_gray = clahe.apply(img_gray)
    T_ci_gt = getPoseFromChessBoard(img_gray, k, dist, squareSize)
    if np.linalg.norm(T_ci_gt) == 0:
        #print("Checkerboard not found")
        return -1, -1
    t =round(tStamp*1e-9, 6)
    print("Image : ", tStamp) 
    ind_df = df_traj[0].sub(t).abs().idxmin()
    
    #ind_df = (df_traj[0].tolist().index(t))
    print("index : ", ind_df)
    quat = np.array(df_traj.iloc[ind_df, 4:8])
    rot = R.from_quat(quat).as_matrix()
    trans = np.array(df_traj.iloc[ind_df,1:4]).reshape(3,1)
    T_0i = np.vstack((np.hstack((rot, trans)), np.array([0,0,0,1])))
    ## In Tum trajectory we have T_0i, pose WRT start, 
    ## GT = T_ci, expected T_ci^ = T_c0 * T_0i=
    print("GT Pose: ")
    print(T_ci_gt)
    if T_ci_gt[3,3] > 2.0 :
        print("calibrationj board is too far. estimate wont be accurate")
        return -1, -1
    print("expected Pose Traj: ")
    T_ci_exp = T_c0 @ T_0i
    print(T_ci_exp)
    #find delta pose
    deltapose = np.linalg.inv(T_ci_exp) @ T_ci_gt
    rotvec_cv,_ = cv2.Rodrigues(deltapose[0:3,0:3])
    trans_error = np.linalg.norm(T_ci_exp[0:3, 3] - T_ci_gt[0:3, 3])
    rot_error = np.rad2deg(np.linalg.norm(rotvec_cv))
    print("translation error: ",trans_error, )
    print("rotation error in degrees: ", rot_error)
    return trans_error, rot_error

In [15]:
def main(data_path,traj_path, k, dist, first_stamp ):
    



    ## read the tum format trajectory file
    df = pd.read_csv(traj_path, header=None,skipinitialspace=True, sep=' ')
    image_path = os.path.join(data_path,str(first_stamp)+".jpg")
    print("image_path" , image_path)
    img_first = cv2.imread(os.path.join(data_path,str(first_stamp)+".jpg"), 0)

    T_cs = getPoseFromChessBoard(img_first, k, dist, 0.081)
    # print(T_cs)

    computeError(df, first_stamp, k, dist, 0.081, (9,9), T_cs)

    imgnames = os.listdir(data_path)
    imgnames.sort()
    imgstamps = np.array([ float(img[:-4])*1e-9 for img in imgnames])



    avg_trans_e = 0
    avg_rot_e = 0
    cnt =0


    for stamp in df[0]:
        if stamp > (first_stamp*1e-9 + 170):
            print("STamp:",stamp)
            diffs = np.absolute(imgstamps - stamp)
            image_index = diffs.argmin()
            trans_e, rot_e = computeError(df, int(imgnames[image_index][:-4]), k, dist, 0.081, (9,9), T_cs)
            if trans_e != -1 and rot_e != -1:
                avg_trans_e = avg_trans_e + trans_e
                avg_rot_e = avg_rot_e + rot_e
                cnt = cnt +1
    print("Average translation error : ", avg_trans_e/cnt)
    print("Average rotational error : ", avg_rot_e/cnt)




In [19]:
# isec_ground1

data_path="/media/neural/pushyami_disk/oldneufr_data/04_21_2022_isec_curry/ISEC_Ground1/cam1"


k = np.array([[890.3108042354397, 0, 371.6746843591246],
             [0, 890.8454719023792, 275.10004314234504],
            [0, 0, 1]])

dist = np.array([-0.21075936736896436, 0.16323979280198103, 5.060027981629671e-06, 8.671705366115946e-05])
first_stamp = 1650574403355778934

traj_path_15 = "/home/neural/multicamSLAM/results/isec_ground1/traj_15.txt"
traj_path_1 = "/home/neural/multicamSLAM/results/isec_ground1/traj_1.txt"
traj_path_156 = "/home/neural/multicamSLAM/results/isec_ground1/traj_156.txt"
traj_path_015 = "/home/neural/multicamSLAM/results/isec_ground1/traj_015.txt"

# main(data_path,traj_path_15, k, dist, first_stamp )

# main(data_path,traj_path_1, k, dist, first_stamp )

# main(data_path,traj_path_156, k, dist, first_stamp )

# main(data_path,traj_path_015, k, dist, first_stamp )



image_path /media/neural/pushyami_disk/oldneufr_data/04_21_2022_isec_curry/ISEC_Ground1/cam1/1650574403355778934.jpg
corners found
corners found
Image :  1650574403355778934
index :  0
GT Pose: 
[[ 0.99323903  0.00404174 -0.11601678  1.19108321]
 [-0.01663858  0.99403162 -0.107816    0.32351058]
 [ 0.11488858  0.10901741  0.98737825 -3.84084279]
 [ 0.          0.          0.          1.        ]]
expected Pose Traj: 
[[ 0.99323903  0.00404174 -0.11601678  1.19108321]
 [-0.01663858  0.99403162 -0.107816    0.32351058]
 [ 0.11488858  0.10901741  0.98737825 -3.84084279]
 [ 0.          0.          0.          1.        ]]
translation error:  0.0
rotation error in degrees:  0.0
STamp: 1650574573.402195
Corners not found
STamp: 1650574573.703099
Corners not found
STamp: 1650574574.153624
Corners not found
STamp: 1650574574.603385
Corners not found
STamp: 1650574575.053067
Corners not found
STamp: 1650574575.502898
Corners not found
STamp: 1650574575.802951
Corners not found
STamp: 1650574576

In [26]:
# curry

data_path="/media/neural/pushyami_disk/oldneufr_data/04_21_2022_isec_curry/Curry_center/cam1"


k = np.array([[890.3108042354397, 0, 371.6746843591246],
             [0, 890.8454719023792, 275.10004314234504],
            [0, 0, 1]])

dist = np.array([-0.21075936736896436, 0.16323979280198103, 5.060027981629671e-06, 8.671705366115946e-05])
first_stamp = 1650576195148245580

# traj_path_15 = "/home/neural/multicamSLAM/results/isec_ground1/traj_15.txt"
# traj_path_1 = "/home/neural/multicamSLAM/results/isec_ground1/traj_1.txt"
# traj_path_156 = "/home/neural/multicamSLAM/results/isec_ground1/traj_156.txt"
traj_path_015 = "/home/neural/multicamSLAM/results/curry/traj_015.txt"

# main(data_path,traj_path_15, k, dist, first_stamp )

# main(data_path,traj_path_1, k, dist, first_stamp )

# main(data_path,traj_path_156, k, dist, first_stamp )

main(data_path,traj_path_015, k, dist, first_stamp )



image_path /media/neural/pushyami_disk/oldneufr_data/04_21_2022_isec_curry/Curry_center/cam1/1650576195148245580.jpg
corners found
corners found
Image :  1650576195148245580
index :  0
GT Pose: 
[[ 9.99934217e-01 -1.06614485e-02  4.23027904e-03  5.37625666e-01]
 [ 1.14191038e-02  9.60033864e-01 -2.79650827e-01  1.12188129e+00]
 [-1.07972825e-03  2.79680736e-01  9.60092454e-01 -4.30040204e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
expected Pose Traj: 
[[ 9.99936295e-01 -1.06346725e-02  3.78273530e-03  5.37533851e-01]
 [ 1.12683133e-02  9.59999139e-01 -2.79776121e-01  1.12185570e+00]
 [-6.56095204e-04  2.79800923e-01  9.60057818e-01 -4.30040947e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
translation error:  9.560303885336382e-05
rotation error in degrees:  0.026744555745663375
STamp: 1650576365.352466
Corners not found
STamp: 1650576366.252385
Corners not found
STamp: 1650576367.150114
Corners not found
STamp: 1650576367.600009
C

In [31]:
# isec_ground2

data_path="/media/neural/pushyami_disk/oldneufr_data/04_14_2022_isec_lab_ground/ISEC_Ground2/cam1"


k = np.array([[890.3108042354397, 0, 371.6746843591246],
             [0, 890.8454719023792, 275.10004314234504],
            [0, 0, 1]])

dist = np.array([-0.21075936736896436, 0.16323979280198103, 5.060027981629671e-06, 8.671705366115946e-05])
first_stamp = 1649979690238438035

traj_path_15 = "/home/neural/multicamSLAM/results/isec_ground1/traj_15.txt"
traj_path_1 = "/home/neural/multicamSLAM/results/isec_ground1/traj_1.txt"
traj_path_156 = "/home/neural/multicamSLAM/results/isec_ground1/traj_156.txt"
traj_path_015 = "/home/neural/multicamSLAM/results/isec_ground2/traj_015.txt"

# main(data_path,traj_path_15, k, dist, first_stamp )

# main(data_path,traj_path_1, k, dist, first_stamp )

# main(data_path,traj_path_156, k, dist, first_stamp )

main(data_path,traj_path_015, k, dist, first_stamp )

image_path /media/neural/pushyami_disk/oldneufr_data/04_14_2022_isec_lab_ground/ISEC_Ground2/cam1/1649979690238438035.jpg
corners found
corners found
Image :  1649979690238438035
index :  0
GT Pose: 
[[ 9.83036455e-01  4.48922014e-03  1.83355323e-01 -3.95649201e-01]
 [-4.23046440e-03  9.99989427e-01 -1.80235810e-03 -7.37243429e-02]
 [-1.83361476e-01  9.96105556e-04  9.83045053e-01 -5.39168048e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
expected Pose Traj: 
[[ 9.83067520e-01  4.50323408e-03  1.83188349e-01 -3.95640606e-01]
 [-4.23644394e-03  9.99989319e-01 -1.84769144e-03 -7.37325998e-02]
 [-1.83194713e-01  1.04033828e-03  9.83076098e-01 -5.39155569e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
translation error:  0.00012535272335282996
rotation error in degrees:  0.010076785714937905
STamp: 1649979861.087118
Corners not found
STamp: 1649979861.687124
Corners not found
STamp: 1649979862.737607
Corners not found
STamp: 1649979863.63

ZeroDivisionError: division by zero